# Notebook auxiliar

In [1]:
import json
import torch
import torch.utils.data as data
import torchaudio
from matplotlib import pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


#### La función `collate_fn` (para construir minibatches con datos de diferente longitud)

In [2]:
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader

class VariableLengthDataset(Dataset):
    def __init__(self): # Inventamos un dataset de longitud variable
        self.data = [
            torch.randint(0, 10, (length,)) for length in [5, 10, 8, 6, 12]
        ]
        self.labels = torch.randint(0, 2, (len(self.data),))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

def collate_fn(batch):
    # El batch es una lista de tuplas: [(dato1,label1), (dato2,label2),...]
    sequences, labels = zip(*batch) # Esto devuelve: 
                                    # sequences = (dato1,dato2,...)
                                    # labels = (label1,label2,...)
    padded_sequences = pad_sequence(sequences, batch_first=True, padding_value=0)
    labels = torch.tensor(labels)
    return padded_sequences, labels # Esta es la salida del dataloader

dataset = VariableLengthDataset()
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

for i in range(len(dataset)):
    dato,label =  dataset[i]
    print(dato,label)
print()

for batch_idx, (padded_sequences, labels) in enumerate(dataloader):
    print(f"Batch {batch_idx + 1}:")
    print("Padded Sequences:")
    print(padded_sequences)
    print("Labels:")
    print(labels)
    print("Shape of padded sequences:")
    print(padded_sequences.shape)

tensor([2, 0, 9, 8, 8]) tensor(1)
tensor([5, 1, 3, 3, 8, 2, 3, 1, 0, 0]) tensor(1)
tensor([6, 0, 5, 8, 8, 6, 2, 8]) tensor(1)
tensor([5, 6, 1, 3, 8, 6]) tensor(1)
tensor([7, 8, 3, 3, 4, 9, 6, 7, 2, 8, 7, 1]) tensor(0)

Batch 1:
Padded Sequences:
tensor([[2, 0, 9, 8, 8, 0, 0, 0, 0, 0],
        [5, 1, 3, 3, 8, 2, 3, 1, 0, 0]])
Labels:
tensor([1, 1])
Shape of padded sequences:
torch.Size([2, 10])
Batch 2:
Padded Sequences:
tensor([[6, 0, 5, 8, 8, 6, 2, 8],
        [5, 6, 1, 3, 8, 6, 0, 0]])
Labels:
tensor([1, 1])
Shape of padded sequences:
torch.Size([2, 8])
Batch 3:
Padded Sequences:
tensor([[7, 8, 3, 3, 4, 9, 6, 7, 2, 8, 7, 1]])
Labels:
tensor([0])
Shape of padded sequences:
torch.Size([1, 12])


In [3]:
for i in range(len(dataset)):
    dato,label =  dataset[i]
    print(dato,label)

tensor([2, 0, 9, 8, 8]) tensor(1)
tensor([5, 1, 3, 3, 8, 2, 3, 1, 0, 0]) tensor(1)
tensor([6, 0, 5, 8, 8, 6, 2, 8]) tensor(1)
tensor([5, 6, 1, 3, 8, 6]) tensor(1)
tensor([7, 8, 3, 3, 4, 9, 6, 7, 2, 8, 7, 1]) tensor(0)


##### Sobre `zip` y el operador `*`

In [4]:
# Si pasamos una lista como argumento a una función y le aplicamos el operador * a dicha lista,
# el operador "desarma" la lista y la convierte en argumentos separados para la función
x = ['a','b', 'c']
print(*x)
print(x[0],x[1],x[2])

# zip es un iterador de tuplas:
# tuplas de entrada: [(a,1),(b,2),(c,3)]
# tuplas de salida: ([a,b,c),(1,2,3)]
l = [('x',1),('y',2),('z',3)]
ll = zip(*l)
for i in ll:
    print(i)

a b c
a b c
('x', 'y', 'z')
(1, 2, 3)


##### Sobre tteradores e iterables
  - Un iterable es cualquier objeto que se pueda recorrer o *iterar*, por ejemplo una lista, una tupla, un rango, un diccionario. 
  - Un iterador es un objeto que implementa el protocolo de *iterador*, es decir, un objeto que tiene definidos los métodos:
    - `__iter__` que devuelve el propio objeto iterador
    - `__next__` que devuelve el siguiente elemento de la secuencia y un `StopIteration` cuando la secuencia llegó a su fin
  - La estructura de control `for`:
    - recibe un iterable, por ejemplo `range(10)`
    - le aplica `iter()` con lo cual lo convierte en iterador
    - llama repetidamente a `next()` hasta que aparezca la excepción `StopIteration` y termina
     

In [1]:
L = [1,2,3,4,5] # L es un iterable
iter_L = iter(L) # iter_L es un iterador
next(iter_L) # Devuelve el primer elemento del iterador

1

##### Sobre los generadores
Un generador es un objeto de tipo iterador creado por:
  - *Una función generadora*. Es decir, una función que devuelve sus resultados usando `yield` en lugar de `return`. Esto hace que la función recuerde el estado cuando se la deja y lo retome cuando se la llame de nuevo. 
  - *Una expresión generadora*. Es igual que la comprehension list solo que se usan paréntesis en lugar de corchetes.

Ojo, no todo objeto de tipo iterador es un objeto generador, solo los que son creados por una función generadora.

También se podría crear un generador como instancia de una clase siempre que dicha tenga implementado los métodos `__iter__` y `__next__` para convertirla en iterador y los elementos del objeto sean creados con un método generador, es decir una función de que devuelva mediante `yield`.


In [ ]:
# Función generadora
def even_numbers(limit):
    num = 0
    while num < limit:
        yield num
        num += 2
gen_even = even_numbers(10) # gen_even es un generador
for n in even_numbers(10):
    print(n)

print(next(gen_even),next(gen_even)) # Devuelve el siguiente elemento del generador


0
2
4
6
8
0 2


In [ ]:
# Expresión generadora
gen_even = (num for num in range(10) if num % 2 == 0) 
for n in gen_even:
    print(n)


0
2
4
6
8


In [8]:
# Clase que se comporta como un iterador
class NumerosCuadrados:
    def __init__(self, limite):
        self.limite = limite
        self.actual = 0 # Maneja el estado

    def __iter__(self):
        return self # Un iterador es un iterable que se devuelve a sí mismo

    def __next__(self):
        if self.actual < self.limite:
            valor = self.actual * self.actual
            self.actual += 1 # Actualiza el estado para la próxima llamada
            return valor
        else:
            raise StopIteration

# Uso de la clase iterador
cuadrados_obj = NumerosCuadrados(5)

for num in cuadrados_obj:
    print(num)

print(type(cuadrados_obj)) # Salida: <class '__main__.NumerosCuadrados'>

0
1
4
9
16
<class '__main__.NumerosCuadrados'>


In [ ]:
# Clase que contiene un método generador
# Para iterar con una instancia de esta 
# clase no es necesario implementar los métodos __iter__ y __next__
# porque el método generador ya lo hace implícitamente ya que todo generador es un iterador.
class SecuenciaPersonalizada:
    def __init__(self, inicio, fin):
        self.inicio = inicio
        self.fin = fin

    def generar_rango(self): # Esto es un método generador
        actual = self.inicio
        while actual <= self.fin:
            yield actual
            actual += 1

# Uso de la clase con un método generador
mi_secuencia = SecuenciaPersonalizada(1, 5)

# Llamar al método generador para obtener un objeto generador
gen_obj = mi_secuencia.generar_rango()

print(type(gen_obj)) # Salida: <class 'generator'>

for num in gen_obj:
    print(num)

<class 'generator'>
1
2
3
4
5


##### Diferencia entre generadores y clases/instancias
  - Clases/Instancias: Mantienen el estado de los datos (variables de instancia) a través de múltiples llamadas a sus métodos. Los métodos, a menos que usen yield, se ejecutan de forma completa cada vez.
  - Generadores: Mantienen el estado de la ejecución (variables locales, punto de pausa) y permiten "pausar" y "reanudar" la ejecución de la función.

Puedes tener métodos de clase que sean generadores (si contienen yield), pero esto es una elección de diseño específica y no una propiedad inherente de todos los métodos de clase.

Recordar que `__iter__` siempre debe devolver un objeto de tipo iterador. En el siguiente ejemplo esto lo hacemos devolviendo con `yield` ya que de este modo se convierte a `__iter__` en una función generadora y por lo tanto devolverá un iterador (de tipo generador)

In [13]:
a = [1,2,3,4,5]
class Prueba():
    def __init__(self, x):
        self.x = a

    def __iter__(self):
    #    return self.x
        for i in self.x:
            yield  -i
p = Prueba(a)

for i in p:
    print(i)  # Salida: 1, 2, 3, 4, 5

-1
-2
-3
-4
-5


##### La función `stack` (unión de tensores)

In [ ]:
import torch 
  
# creating tensors 
x = torch.tensor([1.,3.,6.,10.]) 
y = torch.tensor([2.,7.,9.,13.]) 
# printing above created tensors 
print("Tensor x:", x) 
print("Tensor y:", y) 
  
# join above tensor using "torch.stack()" 
print("join tensors:") 
t = torch.stack((x,y)) 
  
# print final tensor after join 
print(t) 


print("join tensors dimension 0:") 
t = torch.stack((x,y), dim = 0) 
print(t) 
  
print("join tensors dimension 1:") 
t = torch.stack((x,y), dim = 1) 
print(t) 

#### Reproducibilidad de los experimentos
Como fijar la semilla y aplicarla a todas las posibles variaciones aleatorias

In [ ]:
import torch
import numpy as np
import random # También para operaciones random de Python

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed) # Para todas las GPUs
    torch.cuda.manual_seed_all(seed) # Para múltiples GPUs
    np.random.seed(seed)
    random.seed(seed)
    # Algunas operaciones de cuDNN pueden ser no deterministas, se recomienda esto:
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False # Puede hacer que el entrenamiento sea más lento

# Definir la semilla que usaremos
MY_SEED = 42
set_seed(MY_SEED)

# Acá empieza el código de la aplicación

In [2]:
import torch
import torchaudio
import torchaudio.transforms as T
import torchaudio.functional as F
import torch.nn as nn

import jiwer 
# Pruebitas para definir 'cer'
reference = 'chau'
cer_score = F.edit_distance('ciao', reference)/len(reference)
print(cer_score)
jcer_score = jiwer.cer(reference, 'ciao')
print(jcer_score)

# Pero en realidad en este programa uso el wer, porque el GreedyDecoder nos da las salidas en caracteres separados por blancos
# (como si fueran palabras). Y jiwer tiene los dos, así que usamos el wer del jiwer. 
# Después sirve como excusa para ver los alineamientos. 
def cer(pred,ref):
    return(jiwer.wer(ref, pred))
    # return(F.edit_distance(pred, ref)/len(ref))

0.5
0.5


##### Sobre `model.train()`, `model.eval()` y `with torch.no_grad()`
  - Usar `model.train()` cuando entrenamos para que `BatchNormalization` y `dropout` funcionen correctamente
  - Usar `model.eval()` cuando hacemos test o validación.
  - Usar `with torch.no_grad()` es decir no calcular el gradiente dentro de lo que esté en el bloque `with`

In [ ]:
# Training mode
model.train()
# Your training loop
# ...
# Now switch to evaluation mode for validation
model.eval()

with torch.no_grad():  # No gradient calculation for evaluation
    out_data = model(data)

# Don't forget to switch back to training mode!
model.train()


##### Esquema general
  - De alguna manera generamos el dataset que consta de (tensores):
      - (x_train, y_train)
      - (x_valid, y_valid)
      - (x_test, y_test)
  - Lo convertimos en un dataset que es un wrap que permite iterar sobre los datos. Podemos hacerlo nosotros o podemos usar `TensorDataset`
  - Convertimos el dataset en un dataloader que es una versión del dataset separada en batches. Esto lo hacemos con `DataLoader`. Si es necesario, a `DataLoader` les podemos pasar una función `collate_fn` que por ejemplo haga un padding si los datos no son todos de la misma longitud. También puede hacer un shuffle de los datos en cada epoch, lo cual es bueno en el entrenamiento. El bs de validación se puede hacer más grande ya que no necesita calcular gradientes ni hacer back propagation.

  Las siguientes sentencias:
  
    - `loss_fn = nn.CrossEntropyLoss()`
    - `loss_func = torch.nn.functional.cross_entropy`

  Hacen lo mismo, pero loss_fn es un objeto y loss_func es una función. Es un tema de programación estructurada, las dos hacen lo mismo

In [ ]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch import optim

train_ds = TensorDataset(x_train, y_train)
valid_ds = TensorDataset(x_valid, y_valid)
test_ds = TensorDataset(x_test, y_test)
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=bs*2)
test_dl = DataLoader(test_ds, batch_size=bs)

def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)
    opt = optim.SGD(model.parameters(), lr=lr) 

    if opt is not None:
        loss.backward()
        opt.step() #for p in model.parameters(): p -= p.grad * lr
        opt.zero_grad()

    return loss.item(), len(xb)

def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    opt = optim.SGD(model.parameters(), lr=lr) 
    test_loss, correct = 0, 0
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_dl: # Batch de train
            pred = model(xb)
            loss = loss_func(pred, yb)
            loss.backward()
            opt.step() #for p in model.parameters(): p -= p.grad * lr
            opt.zero_grad()
        model.eval()
        with torch.no_grad():
            for xb, yb in valid_dl:
                pred = model(xb)
                test_loss += loss_func(pred,yb).item()

            correct += (pred.argmax(1) == yb).type(torch.float).sum().item()

            losses, nums = zip(
                *[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl]
            )
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

        print(epoch, val_loss)

tensor(1.)

In [ ]:
import torch
a = torch.tensor([0.1,0.7,0.3,0,0.9])
a.argmax().type(float32) == 3

tensor(False)

In [25]:
au = {'key': 'mbwm0_si1934', 'scored': True, 'hyp_absent': False, 'hyp_empty': False, 'num_edits': 7, 
 'num_ref_tokens': 13, 'WER': 53.84615384615385, 'insertions': 0, 'deletions': 7, 'substitutions': 0, 
 'alignment': [(...), (...), (...), (...), (...), (...), (...), (...), (...), (...), (...), (...), (...)], 
 'ref_tokens': [['sil', 'w', 'ey', 'dx', 'ah', 'l', 'ih', 'dx', 'l', 'w', 'ay', 'l', 'sil']], 
 'hyp_tokens': [['sil', 'w', 'ey', 'w', 'l', 'sil']]}
summary = {'WER': 55.38594854019464, 'SER': 100.0, 'num_edits': 8309, 'num_scored_tokens': 15002, 
           'num_erroneous_sents': 400, 'num_scored_sents': 400, 'num_absent_sents': 0, 
           'num_ref_sents': 400, 'insertions': 191, 'deletions': 5415, 
           'substitutions': 2703, 'error_rate': 55.38594854019464}


In [26]:
import speechbrain
cer_stats = speechbrain.utils.metric_stats.ErrorRateStats()
cer_stats.append(ids=['au'], predict = au['hyp_tokens'], target = au['ref_tokens'])
cer_stats.summarize()

{'WER': 53.84615384615385,
 'SER': 100.0,
 'num_edits': 7,
 'num_scored_tokens': 13,
 'num_erroneous_sents': 1,
 'num_scored_sents': 1,
 'num_absent_sents': 0,
 'num_ref_sents': 1,
 'insertions': 0,
 'deletions': 7,
 'substitutions': 0,
 'error_rate': 53.84615384615385}

In [1]:
# Creación  de un archivo JSON

try:
    with open('p.json', 'x') as f:
        f.write("""
{
"mrws1_sx320": {
"wav": "/dbase/timit/test/dr5/mrws1/sx320.wav",
"duration": 3.28325,
"spk_id": "mrws1",
"phn": "sil dh ih n ih r ih s ih n ih sil g aa sil m ey n aa sil b iy w ih th ih n w aa sil k ih ng sil d ih s sil t ih n sil s sil",
"wrd": "the nearest synagogue may not be within walking distance",
"ground_truth_phn_ends": "2360 2840 3216 4511 5556 7018 7880 10440 11160 12040 13160 13960 14200 17640 18280 19160 20360 21560 23800 25320 25720 26520 27800 28825 30440 31248 32208 34130 35880 36760 37640 37960 39101 40120 40360 41640 43000 43320 44200 44440 45280 46680 49560 52480"
},
} """)
except FileExistsError:
    print("p.json already exists. Skipping dummy file creation.")
        


p.json already exists. Skipping dummy file creation.


## Creación de un greedy ctc decoder

In [ ]:
import torch

def greedy_ctc_decode(emissions: torch.Tensor, blank_idx: int) -> list[str]:
    """
    Performs greedy CTC decoding on a batch of log-probabilities.

    Args:
        emissions: Tensor of shape (seq_len, batch_size, num_classes)
                   containing log-probabilities.
        blank_idx: Index of the blank token.

    Returns:
        A list of decoded strings, one for each sequence in the batch.
    """
    decoded_sequences = []
    # Permute to (batch_size, seq_len, num_classes) for easier argmax
    emissions = emissions.permute(1, 0, 2)

    for i in range(emissions.shape[0]): # Iterate over batch
        # Get the index of the max probability at each timestep
        argmax_preds = emissions[i].argmax(dim=-1)

        decoded_seq = []
        last_char_idx = -1
        for char_idx in argmax_preds:
            if char_idx != blank_idx and (char_idx != last_char_idx or last_char_idx == blank_idx):
                # Add if not blank and not a repeated character (unless the last was blank)
                decoded_seq.append(char_idx.item())
            last_char_idx = char_idx

        # Convert indices to actual characters (you'll need your vocab mapping)
        # For this example, let's assume `tokens` list is available
        decoded_strings = [tokens[idx] for idx in decoded_seq]
        decoded_sequences.append("".join(decoded_strings))
    return decoded_sequences

# Example usage
tokens = ["<blank>", "a", "b", "c", " "] # Your actual vocabulary
blank_idx = tokens.index("<blank>")

# Example emissions (seq_len, batch_size, num_classes)
# Let's say the model outputs: "a-a-b-blank-b-c" (where '-' is blank)
# This should decode to "abc"
emissions_example = torch.zeros(7, 1, len(tokens))
emissions_example[0, 0, tokens.index("a")] = 10
emissions_example[1, 0, blank_idx] = 10
emissions_example[2, 0, tokens.index("a")] = 10
emissions_example[3, 0, blank_idx] = 10
emissions_example[4, 0, tokens.index("b")] = 10
emissions_example[5, 0, blank_idx] = 10
emissions_example[6, 0, tokens.index("c")] = 10

# Add a small amount of noise to other classes to avoid all zeros in softmax
emissions_example += torch.randn_like(emissions_example) * 0.1

decoded_greedy = greedy_ctc_decode(emissions_example, blank_idx)
print(f"Greedy decoded: {decoded_greedy}")

## Bloque de CNN, RNN y DNN teniendo en cuenta el padding


In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchaudio
import json
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from collections import OrderedDict
import os
import math

# --- Utilidad: Transpose Layer ---
# Esta clase es necesaria porque tu CNN_block la utiliza para permutar dimensiones
# antes y después de LayerNorm, ya que LayerNorm suele aplicarse sobre la dimensión de características.
class Transpose(nn.Module):
    """
    Un módulo simple para permutar las dimensiones de un tensor.
    Útil para insertar en nn.Sequential donde se necesita una operación de transposición.
    """
    def __init__(self, dim0: int, dim1: int):
        super().__init__()
        self.dim0 = dim0
        self.dim1 = dim1

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return x.transpose(self.dim0, self.dim1)

# --- Tu CNN_block ---
class CNN_block(nn.Module):
    """
    Bloque convolucional 1D con normalización de capa y manejo de padding.
    Diseñado para procesar secuencias de características (ej. Mel-espectrogramas).
    El input esperado es (batch, in_channels, sequence_length).

    Args:
        in_channels (int): Número de canales de entrada (ej. n_mels para el primer bloque).
        out_channels (int): Número de canales de salida.
        kernel_size (int): Tamaño del kernel convolucional (para Conv1d).
        pool_kernel_size (int): Tamaño del kernel para el MaxPool1d (pooling en la dimensión temporal).
        do_prob (float): Probabilidad de dropout.
    """
    def __init__(self, in_channels: int, out_channels: int, kernel_size: int, 
                    pool_kernel_size: int, do_prob: float = 0.0):
        super().__init__()
        layers = []
        
        layers.append( nn.Conv1d(in_channels=in_channels, out_channels=out_channels, 
                                 kernel_size=kernel_size, stride=1, 
                                 padding="same", padding_mode= "replicate") )
        
        layers.append(Transpose(1,2))
        layers.append(nn.LayerNorm(out_channels)) # LayerNorm opera sobre la última dimensión, que es out_channels
        layers.append(Transpose(1,2))
        
        layers.append(nn.LeakyReLU())
        
        layers.append(nn.MaxPool1d(kernel_size=pool_kernel_size, stride=1))
        layers.append(nn.Dropout(p=do_prob))
        self.bloque_cnn = nn.Sequential(*layers)
        self.pool_kernel_size = pool_kernel_size
    
    def forward(self, x: torch.Tensor, lengths: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
        """
        Forward pass para el CNN_block.

        Args:
            x (torch.Tensor): Tensor de entrada (batch, in_channels, sequence_length).
            lengths (torch.Tensor): Tensor de longitudes originales de las secuencias (batch,).
                                    Estas longitudes corresponden a la 'sequence_length' de 'x'.

        Returns:
            tuple[torch.Tensor, torch.Tensor]:
                - x_masked (torch.Tensor): Salida del bloque CNN con padding enmascarado.
                                           Shape: (batch, out_channels, new_sequence_length)
                - new_lengths (torch.Tensor): Longitudes actualizadas de las secuencias
                                              después del pooling. Shape: (batch,).
        """
        # Calcular las nuevas longitudes después del MaxPool1d
        # MaxPool1d con stride=1 reduce la longitud en (kernel_size - 1)
        new_lengths = lengths - self.pool_kernel_size + 1
        
        # Asegurarse de que las longitudes no sean negativas o cero (mínimo 1)
        new_lengths = torch.clamp(new_lengths, min=1) 

        x = self.bloque_cnn(x)

        # Crear una máscara booleana para las regiones de padding
        output_max_len = x.size(2)
        
        mask = torch.arange(output_max_len, device=x.device).unsqueeze(0) < new_lengths.unsqueeze(1)
        mask_expanded = mask.unsqueeze(1) 
        
        x_masked = x * mask_expanded.float() # Aplicar la máscara

        return x_masked, new_lengths

# --- Modelo Principal CRDNN ---
class CRDNNModel(nn.Module):
    """
    Modelo tipo CRDNN (Convolutional Recurrent Deep Neural Network)
    que integra extracción de características Mel, bloques CNN, RNN y DNN.
    Gestiona la propagación de longitudes para el padding a lo largo de la red.
    """
    def __init__(self, 
                 sample_rate: int,
                 n_fft: int,
                 hop_length: int,
                 n_mels: int, # Dimensión de salida del MelSpectrogram y entrada al primer CNN_block
                 n_cnn_blocks: int, 
                 cnn_channels: list[int], # Lista de canales de salida para cada CNN_block
                 cnn_kernel_size: int, 
                 cnn_pool_kernel_size: int, 
                 cnn_dropout: float,
                 n_rnn_layers: int, 
                 rnn_hidden_size: int, 
                 rnn_bidirectional: bool, 
                 rnn_dropout: float,
                 n_dnn_layers: int, 
                 dnn_hidden_size: int, 
                 dnn_dropout: float,
                 num_classes: int # Número de clases de salida (ej. fonemas + CTC blank)
                ):
        super().__init__()

        self.sample_rate = sample_rate
        self.hop_length = hop_length

        # --- Extracción de Características: MelSpectrogram ---
        # torchaudio.transforms.MelSpectrogram espera input (..., time_samples)
        # y produce output (..., n_mels, time_frames)
        self.mel_transform = torchaudio.transforms.MelSpectrogram(
            sample_rate=sample_rate,
            n_fft=n_fft,
            hop_length=hop_length,
            n_mels=n_mels,
            normalized=True # Normaliza la energía para evitar valores muy grandes
        )

        # --- Bloques CNN ---
        self.n_cnn_blocks = n_cnn_blocks
        if not isinstance(cnn_channels, list) or len(cnn_channels) != n_cnn_blocks:
            raise ValueError(
                f"cnn_channels debe ser una lista con {n_cnn_blocks} elementos, "
                f"pero se recibió: {cnn_channels} (longitud {len(cnn_channels) if isinstance(cnn_channels, list) else 'N/A'})"
            )

        cnn_modules = nn.ModuleList()
        current_in_channels = n_mels # La entrada al primer CNN_block es la salida de MelSpectrogram
        
        for i in range(n_cnn_blocks):
            out_channels = cnn_channels[i]
            
            cnn_modules.append(
                CNN_block(
                    in_channels=current_in_channels,
                    out_channels=out_channels,
                    kernel_size=cnn_kernel_size,
                    pool_kernel_size=cnn_pool_kernel_size,
                    do_prob=cnn_dropout
                )
            )
            current_in_channels = out_channels
        self.cnn_blocks = cnn_modules

        # --- Preparación para RNN ---
        # La salida del último CNN_block es (batch, final_cnn_channels, final_time_frames)
        # La RNN espera (batch, final_time_frames, features_dim)
        rnn_input_size = cnn_channels[-1] 

        # --- Bloques RNN (GRU en este caso) ---
        self.n_rnn_layers = n_rnn_layers
        rnn_modules = nn.ModuleList()
        current_rnn_input_size = rnn_input_size
        
        for i in range(n_rnn_layers):
            rnn_modules.append(
                nn.GRU(
                    input_size=current_rnn_input_size,
                    hidden_size=rnn_hidden_size,
                    num_layers=1, # Cada GRU en la lista es una sola capa
                    batch_first=True,
                    bidirectional=rnn_bidirectional,
                    dropout=rnn_dropout if i < n_rnn_layers - 1 else 0 
                )
            )
            current_rnn_input_size = rnn_hidden_size * 2 if rnn_bidirectional else rnn_hidden_size
        self.rnn_blocks = rnn_modules

        # --- Bloques DNN ---
        self.n_dnn_layers = n_dnn_layers
        dnn_modules = nn.ModuleList()
        current_dnn_input_size = rnn_hidden_size * 2 if rnn_bidirectional else rnn_hidden_size 
        
        for i in range(n_dnn_layers):
            out_dnn_size = dnn_hidden_size
            dnn_modules.append(
                nn.Sequential(
                    nn.Linear(current_dnn_input_size, out_dnn_size),
                    nn.LeakyReLU(), 
                    nn.Dropout(p=dnn_dropout)
                )
            )
            current_dnn_input_size = out_dnn_size
        self.dnn_blocks = dnn_modules

        # --- Capa de Salida Final ---
        self.output_layer = nn.Linear(current_dnn_input_size, num_classes)

    def forward(self, x_waveform: torch.Tensor, lengths_waveform: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
        """
        Forward pass del modelo CRDNN.

        Args:
            x_waveform (torch.Tensor): Tensor de entrada de audio crudo (batch, num_samples).
            lengths_waveform (torch.Tensor): Tensor de longitudes originales de las muestras de audio (batch,).

        Returns:
            tuple[torch.Tensor, torch.Tensor]:
                - logits (torch.Tensor): Salidas del modelo (batch, final_time_frames, num_classes).
                - output_lengths (torch.Tensor): Longitudes de las secuencias de salida (batch,).
        """
        # --- 1. Extracción de Características Mel-espectrograma ---
        # x_waveform es (batch, num_samples)
        # mel_features será (batch, n_mels, time_frames)
        mel_features = self.mel_transform(x_waveform)

        # Calcular las longitudes de los frames del Mel-espectrograma
        # La longitud de los frames es floor((num_samples - n_fft) / hop_length) + 1
        # O, si MelSpectrogram maneja padding, puede ser ceil(num_samples / hop_length)
        # Usaremos la fórmula de torchaudio para calcular las longitudes de los frames:
        # num_frames = floor( (num_samples - n_fft) / hop_length ) + 1
        # Sin embargo, torchaudio.transforms.MelSpectrogram puede tener un comportamiento de padding
        # que hace que la longitud sea ceil(num_samples / hop_length).
        # Para ser robustos, usaremos la longitud real de la dimensión temporal de mel_features.
        # Y para las longitudes, necesitamos calcular cuántos frames válidos hay.
        
        # Una forma común de calcular las longitudes de los frames es:
        # mel_frame_lengths = (lengths_waveform.float() / self.hop_length).ceil().long()
        # Esto puede ser una aproximación. La forma más precisa es usar la longitud real de la dimensión temporal
        # y ajustar las longitudes originales de los samples.
        
        # La longitud de los frames después de MelSpectrogram es:
        # (longitud_original_samples - n_fft) / hop_length + 1
        # O, si hay `pad_to_max_len=True` en MelSpectrogram, es más complejo.
        # Para la mayoría de los casos, la longitud de los frames se reduce por `hop_length`.
        
        # Calculamos la longitud de los frames válidos en el Mel-espectrograma
        # Usamos la fórmula de torchaudio para `compute_output_shape` de un STFT
        # num_frames = floor((num_samples - n_fft) / hop_length) + 1
        # Si el MelSpectrogram no hace padding adicional, la longitud del frame es:
        mel_frame_lengths = torch.floor_divide(lengths_waveform - self.mel_transform.n_fft, self.hop_length) + 1
        mel_frame_lengths = torch.clamp(mel_frame_lengths, min=1) # Asegurar que no sea cero o negativo

        # --- 2. Forward Pass de los Bloques CNN ---
        current_x = mel_features
        current_lengths = mel_frame_lengths

        for cnn_block in self.cnn_blocks:
            current_x, current_lengths = cnn_block(current_x, current_lengths)
        
        # current_x es (batch, final_cnn_channels, final_time_frames)
        # current_lengths son las longitudes de los frames después de las CNNs

        # --- 3. Preparación para RNN ---
        # Permutar las dimensiones para que la RNN reciba (batch, time_frames, features_dim)
        rnn_input = current_x.permute(0, 2, 1).contiguous() 

        # --- 4. Forward Pass de los Bloques RNN ---
        # Empaquetar secuencias para manejar el padding en la RNN.
        # `lengths` debe estar en CPU para `pack_padded_sequence`.
        packed_rnn_input = pack_padded_sequence(
            rnn_input, current_lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        
        packed_rnn_output = packed_rnn_input
        for rnn_block in self.rnn_blocks:
            packed_rnn_output, _ = rnn_block(packed_rnn_output)
        
        # Desempaquetar la salida de la RNN para volver a tener un tensor acolchado
        rnn_output, _ = pad_packed_sequence(packed_rnn_output, batch_first=True)
        # rnn_output ahora es (batch, final_time_frames, rnn_output_features_dim)

        # --- 5. Forward Pass de los Bloques DNN ---
        dnn_input = rnn_output
        for dnn_block in self.dnn_blocks:
            dnn_input = dnn_block(dnn_input)
        
        # --- 6. Capa de Salida Final ---
        logits = self.output_layer(dnn_input)

        # Devolver los logits y las longitudes de los frames de salida (current_lengths)
        # Estas longitudes son cruciales para el cálculo de la función de pérdida CTC.
        return logits, current_lengths


# --- CLASES Y FUNCIONES DE DATOS (Para que el ejemplo sea autocontenido y ejecutable) ---

# --- Variables de Ruta ---
vocab_file = 'data/label_encoder_new.txt'
train_json = 'data/train.json'
test_json = 'data/test.json'
valid_json = 'data/dev.json'

# --- Función para Cargar Vocabulario ---
def load_phoneme_vocabulary(filepath: str) -> tuple[dict, dict]:
    phoneme_to_idx = {}
    idx_to_phoneme = {}
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split('=>')
            if len(parts) == 2:
                phoneme = parts[0].strip().strip("'")
                idx_str = parts[1].strip()
                try:
                    index = int(idx_str)
                    phoneme_to_idx[phoneme] = index
                    idx_to_phoneme[index] = phoneme
                except ValueError:
                    raise ValueError(f"Error: Índice inválido en la línea: '{line.strip()}'")
            else:
                raise ValueError(f"Error: Línea mal formada (se esperaba 'fonema=>indice'): '{line.strip()}'")
    return phoneme_to_idx, idx_to_phoneme

# --- Clase TimitDataset ---
class TimitDataset(Dataset):
    def __init__(self, json_file: str, vocab_file: str):
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                self.datos_json = json.load(f)
        except FileNotFoundError:
            raise FileNotFoundError(f"Error: El archivo JSON de datos '{json_file}' no se encuentra. Asegúrate de que la ruta sea correcta.")
        except json.JSONDecodeError:
            raise ValueError(f"Error: El archivo '{json_file}' no es un JSON válido.")
        
        self.datos_ids = list(self.datos_json.keys())
        
        try:
            self.str2int, self.int2str = load_phoneme_vocabulary(vocab_file)
        except FileNotFoundError:
            raise FileNotFoundError(f"Error: El archivo de vocabulario '{vocab_file}' no se encuentra. Asegúrate de que la ruta sea correcta.")
        except ValueError as e:
            raise ValueError(f"Error al cargar el vocabulario: {e}")

    def __len__(self):
        return len(self.datos_ids)

    def __getitem__(self, idx: int) -> tuple[torch.Tensor, torch.Tensor]:
        key = self.datos_ids[idx]
        wav_path = self.datos_json[key]['wav']
        phn_text = self.datos_json[key]['phn']

        try:
            waveform, _ = torchaudio.load(wav_path)
        except FileNotFoundError:
            raise FileNotFoundError(f"Error: Archivo de audio '{wav_path}' no encontrado para la muestra '{key}'.")
        except Exception as e:
            raise RuntimeError(f"Error al cargar el audio '{wav_path}' para la muestra '{key}': {e}")

        if waveform.ndim > 1 and waveform.shape[0] == 1:
            waveform = waveform.squeeze(0) # Asegurarse de que sea 1D (samples,)

        phn_list = [p for p in phn_text.strip().split() if p] 
        phn_indices = [self.str2int[phoneme] for phoneme in phn_list if phoneme in self.str2int]
        
        if not phn_indices and phn_list:
            print(f"Advertencia: No se encontraron índices válidos para los fonemas en '{key}': {phn_list}")
            phn_tensor = torch.tensor([], dtype=torch.long)
        else:
            phn_tensor = torch.tensor(phn_indices, dtype=torch.long)
        
        return waveform, phn_tensor # Devolvemos waveform (raw audio) y phn_tensor

# --- Función collate_fn para DataLoader ---
def collate_fn(batch: list[tuple[torch.Tensor, torch.Tensor]]) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    Función de agrupación (collate_fn) para el DataLoader.
    Acolcha secuencias de audio y etiquetas, y devuelve sus longitudes originales
    y sus longitudes normalizadas (relativas a la longitud máxima del batch).
    """
    # Desempaquetar el batch: waveforms son audio crudo, phn_tensors son etiquetas de fonemas
    waveforms, phn_tensors = zip(*batch)

    # --- Procesar Waveforms (audio crudo) ---
    # 1. Longitudes originales de las waveforms (en número de samples)
    waveform_lengths_orig = torch.tensor([w.shape[0] for w in waveforms], dtype=torch.long)
    
    # 2. Acolchar waveforms
    padded_waveforms = pad_sequence(waveforms, batch_first=True, padding_value=0.0)
    
    # 3. Longitud máxima del batch para waveforms (después de acolchar)
    max_waveform_len = padded_waveforms.shape[1]
    
    # 4. Longitudes normalizadas (relativas a la longitud máxima del batch)
    waveform_lengths_norm = waveform_lengths_orig.float() / max_waveform_len

    # --- Procesar Etiquetas de Fonemas ---
    # 1. Longitudes originales de las etiquetas de fonemas
    phn_lengths_orig = torch.tensor([p.shape[0] for p in phn_tensors], dtype=torch.long)
    
    # 2. Acolchar etiquetas de fonemas
    # Se recomienda usar un padding_value que no sea un índice de fonema real si es posible.
    # Aquí, usaremos 0 por simplicidad, asumiendo que es el ID de 'sil' o un token de padding.
    padded_phns = pad_sequence(phn_tensors, batch_first=True, padding_value=0)
    
    # 3. Longitud máxima del batch para fonemas
    max_phn_len = padded_phns.shape[1]
    
    # 4. Longitudes normalizadas (relativas a la longitud máxima del batch)
    phn_lengths_norm = phn_lengths_orig.float() / max_phn_len

    # Retorna todas las versiones: acolchadas, longitudes originales, longitudes normalizadas
    return padded_waveforms, waveform_lengths_orig, waveform_lengths_norm, \
           padded_phns, phn_lengths_orig, phn_lengths_norm


# --- Ejemplo de Uso Principal ---
if __name__ == "__main__":
    # --- Configuración de Parámetros Dummy ---
    # Estos serían tus hparams cargados de un YAML
    hparams_example = {
        "sample_rate": 16000,
        "n_fft": 400,       # 25ms window at 16kHz
        "hop_length": 160,  # 10ms hop at 16kHz
        "n_mels": 80,       # Número de Mel-bins
        "n_cnn_blocks": 2,
        "cnn_channels": [64, 128], # Canales de salida para cada CNN block
        "cnn_kernel_size": 3,
        "cnn_pool_kernel_size": 2, # Pooling en la dimensión temporal (reduce longitud en 1 por bloque)
        "cnn_dropout": 0.15,
        "n_rnn_layers": 2,
        "rnn_hidden_size": 256,
        "rnn_bidirectional": True,
        "rnn_dropout": 0.15,
        "n_dnn_layers": 3,
        "dnn_hidden_size": 512,
        "dnn_dropout": 0.15,
        "num_classes": 40 # Ejemplo: 39 fonemas + 1 para el blank de CTC
    }

    # --- Preparación de Archivos Dummy para Ejecución ---
    # Crea la carpeta 'data' si no existe
    if not os.path.exists('data'):
        os.makedirs('data')
    dummy_wav_dir = 'data/dummy_wavs'
    if not os.path.exists(dummy_wav_dir):
        os.makedirs(dummy_wav_dir)

    def create_dummy_wav(path, duration_samples, sr=16000):
        dummy_audio = torch.randn(1, duration_samples) * 0.1 # Pequeño volumen
        torchaudio.save(path, dummy_audio, sr)

    # Crea un vocabulario dummy
    with open(vocab_file, 'w', encoding='utf-8') as f:
        f.write("sil=>0\n")
        f.write("sp=>1\n")
        f.write("aa=>2\n")
        f.write("ae=>3\n")
        f.write("z=>4\n")
        f.write("ah=>5\n")
        f.write("bh=>6\n")
        f.write("ch=>7\n")
        f.write("dh=>8\n")
        f.write("eh=>9\n")
        f.write("er=>10\n")
        f.write("f=>11\n")
        f.write("g=>12\n")
        f.write("hh=>13\n")
        f.write("ih=>14\n")
        f.write("iy=>15\n")
        f.write("jh=>16\n")
        f.write("k=>17\n")
        f.write("l=>18\n")
        f.write("m=>19\n")
        f.write("n=>20\n")
        f.write("ng=>21\n")
        f.write("ow=>22\n")
        f.write("oy=>23\n")
        f.write("p=>24\n")
        f.write("r=>25\n")
        f.write("s=>26\n")
        f.write("sh=>27\n")
        f.write("t=>28\n")
        f.write("th=>29\n")
        f.write("uh=>30\n")
        f.write("uw=>31\n")
        f.write("v=>32\n")
        f.write("w=>33\n")
        f.write("y=>34\n")
        f.write("zh=>35\n")
        f.write("dx=>36\n")
        f.write("nx=>37\n")
        f.write("eng=>38\n")
        f.write("blank=>39\n") # Para el token blank de CTC

    # Crea archivos JSON dummy para train, test, dev
    # Asegúrate de que las duraciones sean suficientes para que los audios no sean demasiado cortos
    # para el n_fft y hop_length definidos.
    sample_data = {
        "sample_001": {"wav": os.path.join(dummy_wav_dir, "audio_001.wav"), "duration": 1.5, "phn": "aa sp ae z"}, # 1.5s * 16000 = 24000 samples
        "sample_002": {"wav": os.path.join(dummy_wav_dir, "audio_002.wav"), "duration": 2.0, "phn": "sil aa ae ah"}, # 2.0s * 16000 = 32000 samples
        "sample_003": {"wav": os.path.join(dummy_wav_dir, "audio_003.wav"), "duration": 0.8, "phn": "z"}, # 0.8s * 16000 = 12800 samples
        "sample_004": {"wav": os.path.join(dummy_wav_dir, "audio_004.wav"), "duration": 2.5, "phn": "aa ae sp sil ah z"}, # 2.5s * 16000 = 40000 samples
    }

    for key, val in sample_data.items():
        wav_path = val['wav']
        duration_samples = int(val['duration'] * hparams_example["sample_rate"])
        create_dummy_wav(wav_path, duration_samples, hparams_example["sample_rate"])

    with open(train_json, 'w', encoding='utf-8') as f:
        json.dump(sample_data, f, indent=4)
    with open(test_json, 'w', encoding='utf-8') as f:
        json.dump(sample_data, f, indent=4)
    with open(valid_json, 'w', encoding='utf-8') as f:
        json.dump(sample_data, f, indent=4)

    print("Archivos dummy creados para la demostración.\n")

    # --- Instanciar y Probar el Modelo CRDNN ---
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Usando dispositivo: {device}\n")

    model = CRDNNModel(**hparams_example).to(device)
    print("Estructura del Modelo CRDNN:")
    print(model)

    print("\n--- Probando un batch a través del modelo ---")
    try:
        train_ds = TimitDataset(train_json, vocab_file)
        train_dl = DataLoader(train_ds, batch_size=2, shuffle=True, collate_fn=collate_fn)

        for i, (waveforms, wf_lengths_orig, wf_lengths_norm, phns, phn_lengths_orig, phn_lengths_norm) in enumerate(train_dl):
            # Mover datos al dispositivo
            waveforms = waveforms.to(device)
            wf_lengths_orig = wf_lengths_orig.to(device)
            # phns y phn_lengths_orig/norm se usarían en la función de pérdida (CTC), no en el forward del modelo

            # Pasar los waveforms y sus longitudes originales al modelo
            logits, output_lengths = model(waveforms, wf_lengths_orig)

            print(f"Batch {i+1}:")
            print(f"  Input Waveforms Shape: {waveforms.shape}")
            print(f"  Input Waveform Lengths (Original): {wf_lengths_orig}")
            print(f"  Logits Shape (salida del modelo): {logits.shape}")
            print(f"  Output Lengths (después de Mel, CNNs y RNNs): {output_lengths}")
            print("-" * 30)
            if i == 0: # Solo mostrar el primer batch para brevedad
                break
        
        print("\n¡Modelo CRDNN integrado y probado con éxito!")

    except Exception as e:
        print(f"\nOcurrió un error durante la ejecución del ejemplo del modelo: {e}")


Archivos dummy creados para la demostración.

Usando dispositivo: cuda



/home/cestien/anaconda3/lib/python3.12/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.15 and num_layers=1
  warnings.warn(


Estructura del Modelo CRDNN:
CRDNNModel(
  (mel_transform): MelSpectrogram(
    (spectrogram): Spectrogram()
    (mel_scale): MelScale()
  )
  (cnn_blocks): ModuleList(
    (0): CNN_block(
      (bloque_cnn): Sequential(
        (0): Conv1d(80, 64, kernel_size=(3,), stride=(1,), padding=same, padding_mode=replicate)
        (1): Transpose()
        (2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (3): Transpose()
        (4): LeakyReLU(negative_slope=0.01)
        (5): MaxPool1d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
        (6): Dropout(p=0.15, inplace=False)
      )
    )
    (1): CNN_block(
      (bloque_cnn): Sequential(
        (0): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=same, padding_mode=replicate)
        (1): Transpose()
        (2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (3): Transpose()
        (4): LeakyReLU(negative_slope=0.01)
        (5): MaxPool1d(kernel_size=2, stride=1, padding=0, dilati

#### Iteraciones con tqdm

In [ ]:
import tqdm
i = 0
for batch in tqdm.tqdm(train_dl,colour="yellow",desc='entrenando'):
    i += 1